# Nanodegree Udacity - Capstone Project - Data Engineering 
### Building a Data Warehouse to analyse immigration, climate, and demographic data

#### Project Summary
The key subject of this project iss to create an ETL pipeline for I94 immigration, global land temperatures and US demographics datasets to form an analytics database on immigration events. A use case for this analytics database is to find immigration patterns. The main element is to enable a [singe-point-of-truth architecure](https://en.wikipedia.org/wiki/Single_source_of_truth) based on the data warehouse.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Import Libaries

Section to import all relevant libaries and to inalize the installations for this project

In [ ]:
# Do all imports and installs here
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import configparser
import datetime as dt 

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import *

import plotly.plotly as py 
import plotly.graph_objs as go 
import requests
requests.packages.urllib3.disable_warnings()

import utility
import etl_functions

import importlib
importlib.reload(utility)
from utility import visualize_missng_values, clean_immigration, clean_temperature_data
from utility import clean_demographics_data, print_formatted_float

### Configuration Data - Loading Process

Loading all access keys, which are necessary to initiate the AWS infrastructure

In [ ]:
config = configparser.ConfigParser()
config.read('Capstone Config.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

### Creation of the SPARK session

The SPARK session has to be initiated to start the large-scale data processing.

In [ ]:
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enbableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

### Scope of the project

This capstone project will focus on three major datasets which will be used to create a data warehouse including respective fact and dimension tables.

* Used Datasets

    1. [US Cities: Demographics](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
    2. [Climate Change: Earth Surface Temperature Data](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data)
    3. [I94 Immigration Data](https://travel.trade.gov/research/reports/i94/historical/2016.html)

* Tools to analyze the data

    1. Python: as programming language - especially to process the data
    2. PySpark: used for large datasets to process the data
    3. Pandas: data analysis for small data sets
    4. AWS S3: used for plain data storage
    5. AWS Redshift: used for data warehousing and data analysis

### Collect the relevant data

The follwoing data sets are recommended and given to fulfil the project goal:

| Data Set | Format / Data Type | Description |
| ---      | ---                | ---         |
| [US Cities: Demographics](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) | CSV (Comma Separated Value) | This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. This data comes from the US Census Bureau's 2015 American Community Survey. |
| [Climate Change: Earth Surface Temperature Data](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data) | CSV (Comma Separated Value) | Provided by Kaggle, this data set points out the average temperature of significant cities and metropoles all over the world. |
| [I94 Immigration Data](https://travel.trade.gov/research/reports/i94/historical/2016.html) | SAS (Statistical Analysis Software) | The US Immigration Center provides information about the arrivals of international travelers focussing on different global regions and countries. Furthermore, it shows data about the visa type, transportation mode, groups of age, visited states, and the top ports of entry. |

## Loading the relevant data - I94 Immigration

This data comes from the US National Tourism and Trade Office. In the past all foreign visitors to the U.S. arriving via air or sea were required to complete paper Customs and Border Protection Form I-94 Arrival/Departure Record or Form I-94W Nonimmigrant Visa Waiver Arrival/Departure Record and this dataset comes from this forms.

*I94 Immigration data* is the foundation of the data warehouse and is structuted as a repository for all customers. The dataset is strcutured for the year 2016 and is devided by month. The folder is located at `../../data/18-83510-I94-Data-2016`. Based on the month the data is stoted in a SAS binary format called *sas7bdat*.

The following step is loading the revelant data for *April 2016*:

In [5]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_df = spark.read.format('com.github.saurfang.sas.spark').load(fname)

### Quality Check - Showing first fives rows of dataset I94 Immigration

In [6]:
immigration_df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


### Quality Check - Number of Records

Counting the total number of loaded records in the data warehouse:

In [ ]:
print_formatted_float(immigration_df.count())

### Quality Check - Drop Duplicates

Focussing on the attribute *visapost*, showing the first 10 rows of filtered duplpicates:

In [ ]:
immigration_df.select("visapost").dropDuplicates().show(10)

### Data Dictionary for dataset - I94 Immigration

[Data Dictionary I94 Immigration](https://github.com/KCvW/DataEng/blob/main/Capstone/Data%20Dictionary%20I94%20Immigration.png)

## Loading the relevant data - World Temperature Data

This dataset is provided by *Kaggle* and shows the temperature for city around the globe:

In [ ]:
# Loading world temperature data
file_name = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_df = spark.read.csv(file_name, header=True, inferSchema=True)

### Quality Check - Showing the first ten rows of dataset World Temperature 

In [ ]:
temperature_df.limit(10).toPandas()

### Data Dictionary for dataset - World Temperature

[Data Dictionary World Temperature](https://github.com/KCvW/DataEng/blob/main/Capstone/Data%20Dictionary%20World%20Temperature.png)

### Quality Check - Number of Records

Counting the total number of records loaded in the data warehouse:

In [ ]:
# verify the total count of records loaded
print_formatted_float(temperature_df.count())

## Loading the relevant data - U.S. City Demographic

*OpenSoft* is providing this dataset. It highlights information about demographic details of U.S. cities and census-deignated places. The threshold in regards to the population is equal or greater to 65,000 citizen. The raw data is provided by the U.S. Census Bureau based on the 2015 American Community Survey.

In [ ]:
# load the dataset
file_name = "us-cities-demographics.csv"
demographics_df = spark.read.csv(file_name inferSchema=True, header=True, sep=';')

### Quality Check - Showing the first ten rows of dataset U.S. City Demographic

In [ ]:
demographics_df.limit(10).toPandas()

### Data Dictionary for dataset - U.S. City Demographic

[U.S. City Demographic](https://github.com/KCvW/DataEng/blob/main/Capstone/Data%20Dictionary%20US%20City%20Demographic.png)

### Quality Check - Number of Records

Counting the total number of records loaded into the data warehouse:

In [ ]:
# verify the total count of records loaded
print_formatted_float(demographics_df.count())

In [ ]:
# not needed - implemented in a further cell
"""
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
"""

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark = spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data - Immigration Data
Exploration and data analysis to check the quality of the raw data sets.
Starting with `Immigration Data`:

In [ ]:
# focussing on a monthly basis all files are listed
files = os.listdir('../../data/18-83510-I94-Data-2016')
files

In [ ]:
# get an overview of the data schema
immigration_df.printSchema()

### Quality Check - Graphical analysis concerning missing data (Immigration Data)
Listing all columns of the data schema to visualze the percentage of missing data per attribute:

In [ ]:
utility.visualize_missing_values_spark(immigration_df)

#### Data Cleaning Process
On  scope are all columns and rows with missing values equal or higher than 90%, because these ones do not contain sufficient data for an reliable analysis approach:

* Deleting all columns with a missing rate greater than 90%
* Deleting all rows with a missing rate of 100%

In [ ]:
# delete all columns with insufficent data quality and values
cols = ['occup', 'entdepu', 'insnum']
new_immi_df = immigration_df.drop(*cols)

In [ ]:
# show new data schema after deletion process
new_immi_df.printSchema()

In [ ]:
# delete all duplicates
new_immi_df = new_immi_df.dropDuplicates(['cicid'])

In [ ]:
# count all entries
print_formatted_float(new_immi_df.count())

In [ ]:
# delete all rows with insufficient data quality and values - no entries
new_immi_df = new_immi_df.dropna(how = 'all', subject = [('cicid')])

In [ ]:
# count after deletion process
print_formatted_float(new_immi_df.count())

In [ ]:
# immigration dataframe after cleansing
new_immigration_df = utility.clean_spark_immigration_data(immigration_df)

### Explore the Data - World Temperature Data
Exploration and data analysis to check the quality of the raw data sets.
Continuing with `World Temperature Data`:

In [ ]:
# show initial data schema
temperature_df.printSchema()

#### Quality Check - Graphical analysis concerning missing data (World Temperature Data)
Listing all columns of the data schema to visualze the percentage of missing data per attribute:

In [ ]:
# column type dt changed to string
temperature_df2 = temperature_df.withColumn("dt", col("dt").cast(StringType()))
utility.visualize_missing_values_spark(temperature_df2)

### Steps to clean the relevant data

To focus on a relibale dataset it is again necessary to purge the data based on the respective values:

* Deletion of rows which have a missing value
* Detection of rows with duplicate values

In [ ]:
# Process of cleansing the data
new_temperture_df = utility.clean_spark_temperature_data(temperature_df)

In [ ]:
# not needed as other procedures are taking care of data quality
"""
# show columns with insufficient data
utility.visualize_missing_values_spark(temperature_df2)
"""

### Explore the data (U.S. City Demographic Data)

The scope is on the raw data set to evalute the quality.

In [ ]:
# counting the number of records in the raw data set
print_formatted_float(demographics_df.count)

In [ ]:
# displaying the data schema
demographics_df.printSchema()

### Showing missing values in the raw dataset

visualization of the raw datasets to get an impression of the entore data quality

In [ ]:
# visualizing the missing attributes
utility.visualize_missing_values_spark(demographics_df)

In [ ]:
# showing columns with no or missing data
nulls_df = pd.DataFrame(data=demographics_df.toPandas().isnull().sum(), columns=['values'])
nulls_df = nulls_df.reset_index()
nulls_df.columns = ['cols', 'values']

In [ ]:
# Calculating the missing in percent (%)
nulls_df['% missing value'] = 100*nulls_df['values']/demographics_df.count()
nulls_df[nulls_df['% missing values']>0]

### Further steps to come to a clean dataset

As mentioned above, also this dataset needs a deepdive to focus on reliable raw data

In [ ]:
# process of cleaning the dataset
new_demographics_df = utility.clean_spark_demographics_data(demographics_df)

### Step 3: Define the Data Model

Coming from a analysis project it is essential to point out the core data model.

#### 3.1 Conceptual Data Model

The conceputal data model this is core to define the main fact table and all related dimension tables:

[Conceptual Data Model](https://github.com/KCvW/DataEng/blob/main/Capstone/Capstone%20Data%20Model%20(conceptual).png)

Coming from this data model it is key to understand how to combine the data to realize relibale and sophisticated analysis approaches. Adapted from the conceptual model the following structure is crucial for the capstone project.

The dimension table `country` combines data reflecting the global city temperatures and information from the U.S. immigration center. Therefore it is possible to calculate statistical combinations liek correlations between both dimensions. 

A similar approach is given for the dimension table `usa_demographics`, which links its own data with the fact table `immigration`. Hence it is possible to extract information about certain immigration pattern such as the combination of U.S. demographics and the population in defined U.S. states. So it is feasible to check whether bigger or ever proser U.S. states attract more poeple in a given time frame. Another interesting approach is to focus on per-defined classes like age, race or even sex. All of these attributes can be visualized in a dashboard to drill down the details or to condens the information. A combination of this data might be helpful for immigration and tourism offices to define strategies for the future.

Another linkage is given by the dimension table `visa_type`. Referring to the primary key `visa_type_key` this table is directly connected to the fact table `immigration` whereas the data is provided by the immigration data set.

As highlighted in the picture `Conceptual Data Model` the fact table `immigration` is the core of the entire data model. All data is provided by the immigration data set and is linked in multiple ways to the mentioned dimension tables. The meta data which is defining the structure is described in the data dictionary.

#### 3.2 Data Pipelines & Data Mapping

In this sub-chapter the scope is on the desciptrion how to create the before-mentioned tables based on the data pipelines.
The primary steps are:

1. Initial load of the entire data sets
2. Process of cleaning `I94 Immigration` - needed to create a monthly data frame in Apache SPARK
3. Creating dimension tables - `visa_type` and `immi_calendar`
4. Extraction and process of cleaning of `World Temperature Data`
5. Create further tables - dimension table `country` and fact table `immigration`
6. Loasding and cleansing the `Demographic` data
7. Creating the dimension table `usa_demographics`

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
The function in the code below creates the dimension table `immi_calendar` adapted from the arrival date. In detail:

* `param df` - Dataframe in Spark listing all immigration events
* `param output_data`- path to where the dimensional dataframe is written to
* `return` - Dataframe in Spark showing the dimensional table `immi_calendar`

In [ ]:
# Creating the dimension table immi_calendar
def create_immigration_calendar_dimension(df, output_data):
    
    # Conversion of arrival date in SAS format to a common datetime object (utf)
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # Creation of data frame for calenmdar based on column arrdate
    calendar_df = df.select(['arrdate']).withColumn("arrdate", get_datetime(df.arrdate)).distinct()
    
    # Extension of data frame by adding further columns
    calendar_df = calendar_df.withColumn('arrival_day', dayofmonth('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_week', weekofyear('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_month', month('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_year', year('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_weekday', dayofweek('arrdate'))
    
    # Creation of an ID in the dataframe calendar
    calendar_df = calendar_df.withColumn('id', monotonically_increasing_id())
    
    # Write the dimensional table to Parquet
    partition_columns = ['arrival_year', 'arrival_month', 'arrival_week']
    calendar_df.write.parquet(output_data + "immigration_calendar", partitionBy=partition_columns, mode="overwrite")
    
    return calendar_df

In [ ]:
output_data = "tables/"

In [ ]:
calendar_df = create_immigration_calendar_dimension(new_immigration_df, output_data)

In this section the dimension table `country` is created.
Similar to the other dimension table the data is merged from `immigration` and `global land temperature` data

* `param df` - Dataframe in Spark listing all immigration events
* `temp df` - Dataframe in Spark listing the global land tempearture data
* `param output_data`- path to where the dimensional dataframe is written to
* `return` - Dataframe in Spark showing the dimensional table `immi_calendar`

In [ ]:
def create_country_dimension_table(df, temp_df, output_data):
    
    # Pulling the temperature data in an aggregated way
    agg_temp = utility.aggregate_temperature_data(temp_df).toPandas()
    
    # Loading the files i94.res.csv to map the country data
    mapping_codes = pd.read_csv('i94res.csv')
    
    @udf('string')
    def get_country_average_temperature(name):
        print("Processing: ", name)
        avg_temp = agg_temp[agg_temp['Country']==name]['average_temperature']
        
        if not avg_temp.empty:
            return str(avg_temp.iloc[0])
        
        return None
    
    @udf()
    def get_country_name(code):
        name = mapping_codes[mapping_codes['code']==code]['Name'].iloc[0]
        
        if name:
            return name.title()
        return None
    
    # selection of i94res columns and renaming these ones
    dim_df = df.select(['i94res']).distinct().withColumnRenamed('i94res', 'country_code')
    
    # Creation of column country_name
    dim_df = dim_df.withColumn('country_name', get_country_name(dim_df.country_code))
    
    # Creation of column average_temperature
    dim_df = dim_df.withColumn('average_temperature', get_country_average_temperature(dim_df.country_name))
    
    # Write the tables to Parquet
    dim_df.write.parquet(output_data + "country", mode="overwrite")
    
    return dim_df

In [ ]:
country_dim_f = create_country_dimension_table(new_immigration_df, new_temperture_df, output_data)

In [ ]:
country_dim_f.show(5)

Next step is to create the dimension table `visa_type`, which is done in the same manner.

* `param df` - Dataframe in Spark listing all immigration events
* `param output_data`- path to where the dimensional dataframe is written to
* `return` - Dataframe in Spark showing the dimensional table `immi_calendar`

In [ ]:
def create_visa_type_dimension_table(df, output_data):
    
    # Creating the dataframe from the column visatype
    visatype_df = df.select(['visatype']).distint()
    
    # Adding a column for the id
    visatype_df = visatype_df.withColumn('visa_type_key', monotonically_increasing_id())
    
    # Write the tables to Parquet
    visatype_df-write-parquet(output_data + "visa_type", mode="overwrite")
    
    return visatype_df

def get_visa_type_dimension(output_data):
    return spark.read.parquet(output_data + "visa_type")

In [ ]:
# create test function for dimension table visa_type
visatype_df = create_visa_type_dimension_table(new_immigration_df, output_data)
visatype_df.show(n=5)

Now the demographics table is created, which is the last dimension table based on the data model. Referring to the common schema the function is nearly the same:

* `param df` - Dataframe in Spark listing all immigration events
* `param output_data`- path to where the dimensional dataframe is written to
* `return` - Dataframe in Spark showing the dimensional table `usa-demographics`

In [ ]:
def create_demographics_dimension_table(df, output_data):
    dim_df =df.withColumnRenamed('Median Age','median_age')\
            .withColumnRenamed('Male Population', 'male_population')\
            .withColumnRenamed('Female Population', 'femaile_population')\
            .withColumnRenamed('Total Population', 'total_population')\
            .withColumnRenamed('Number of Veterans', 'number_of_veterans')\
            .withColumnRenamed('Foreign-born', 'foreign_born')\
            .withColumnRenames('Average Household Size', 'average_household_size')\
            .withColumnRenamed('State Code', 'state_code')
# Adding a column for the ID
    dim_df = dim_df.withColumn('id', monotonically_increasing_id())
# write tables to Parquet
    dim_df.write.parquet(output_data + "demographics", mode="overwrite")
    return dim_df

In [ ]:
demographics_dim_df = create_demographics_dimension_table(new_demographics_df, output_data)
demographics_dim_df.limit(5).toPandas()

Finally the fact table `Immigration` has to be initiated. Also this element is created in a similar manner.

* `param df` - Dataframe in Spark listing all immigration events
* `visa_type_df` - refers to the Spark dataframe listing all the global city temperature data
* `param output_data`- path to where the dimensional dataframe is written to
* `return` - Dataframe in Spark showing the dimensional table `immi-calendar`

In addition a user defined function is used to get the visa key.

* `param visa_type` - visa type U.S. non-immigrant
* `return` - respective visa key

In [ ]:
def create_immigration_fact_table(df, output_data):
    #getting dimension visa_type
    dim_df = get_visa_type_dimension(output_data).toPandas()
    
    @udf('string')
    def get_visa_key(visa_type):
        key_series = dim_df[dim_df['visatype']==visa_type]['visa_type_key']
        
        if not key_series.empty:
            return str(key_series.iloc[0])
        
        return None
    
    # Based on udf the arrivial date is converted from SAS to a datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # Referring to data model certain columns are renamed
    df = df.withColumnRenamed('cicid', 'record_id') \
            .withColumnRenamed('i94res', 'country_residence_code') \
            .withColumnRenamed('i94addr', 'state_code')
            
    # create key visa_type
    df = df.withColumn('visa_type_key', get_visa_key('visatype'))
    
    # Conversion of of arrival date into a datetime object
    df = df.withColumn("arrdate", get_datetime(df.arrdate))
    
    # write fact table to Parquet
    df.write.parquet(output_data + "immigration_fact", mode = "overwrite")
    
    return df

In [ ]:
immigration_fact_df = create_immigration_fact_table(new_immigration_df, output_data)

In [ ]:
# Loading the relevant data
# Operating all function for data cleansing
# Creating all defined dimension and fact tables
def run_pipeline():

#### 4.2 Data Quality Checks

Basically the quality checks are focus two main areas:

1. Does the data schema of the dimensional tables are referring to the data model
2. Avoiding empty tables after the ETL was executed

Please refer to the respective Jupyter Notebook:
[Check Data Quality.ipynb][https://github.com/KCvW/DataEng/blob/main/Capstone/Check%20Data%20Quality.ipynb]

A quick check can be performed by executing this code:

In [ ]:
# Perform quality checks here
table_dfs = {
    'immigration_fact': immigration_fact_df,
    'visa_type_dim': visatype_df,
    'calendar_dim': calendar_df,
    'usa_demographics_dim': demographics_dim_df,
    'country_dim': country_dim_f
}
for table_name, table_df in table_dfs.item():
    etl_functions.quality_checks(table_df, table_name)

#### 4.3 Data dictionary 
The final `Data Dictionary` is based on the star schema which was initially described in the former chapters. The star schema consists of all raw data sources that were used in this capstone project. 

These are the fact and dimension tables created by the ETL process:

* [Fact Table](https://github.com/KCvW/DataEng/blob/main/Capstone/Fact%20Table.png)
Final fact table - includes all necessary features and descriptions

* [Country Dimension Table](https://github.com/KCvW/DataEng/blob/main/Capstone/Country%20Dimension%20Table.png)
Focussing on `country_code` and `country_name` - these fields were extracted from the SAS file - `average_temperature` is gathered from the global land temperature file

* [Visa Type Dimension Table](https://github.com/KCvW/DataEng/blob/main/Capstone/Visa%20Type%20Dimension%20Table.png)
Extracts the attributes `visa_type_key` and `visa_type` just to ensure a relibale reconciliation of the data

* [Immigration Calendar Dimension Table](https://github.com/KCvW/DataEng/blob/main/Capstone/Immigration%20Calendar%20Dimension%20Table.png)
All features were gathered from the initial dataset `immigration`

* [US Demographics Dimension Table](https://github.com/KCvW/DataEng/blob/main/Capstone/US%20Demographics%20Dimension%20Table.png)
Also this data is based on the a single dataset `us_cities_demographics`

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.